In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# Load the dataset
data = pd.read_csv('put/your/path')

# Display basic information about the dataset
print(data.info())
print(data.head())


In [ ]:
# Display a summary of people who have heart disease (target = 1)
plt.figure(figsize=(8, 6))
sns.countplot(x='target', data=data, palette='viridis')
plt.title('Distribution of Heart Disease Patients')
plt.xlabel('Target (0: No Heart Disease, 1: Heart Disease)')
plt.ylabel('Count')
plt.show()

# Fill missing values or drop rows/columns with missing values
data = data.dropna()  # Or you can use data.fillna(method='ffill') or any other strategy

# Encode categorical features
data = pd.get_dummies(data, drop_first=True)

# Separate features and target variable
X = data.drop('target', axis=1)  # Replace 'target' with the actual name of your target column
y = data['target']

# Encode the target labels if necessary
le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Build the model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(np.max(y_train) + 1, activation='softmax')  # Output layer adjusted based on number of unique classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=2)


In [ ]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_test  # Assuming y_test is already integer-encoded

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_true, y_pred_classes))

# Convert le.classes_ to list of strings for classification_report
target_names = list(map(str, le.classes_))
print(classification_report(y_true, y_pred_classes, target_names=target_names))


In [ ]:
# Visualize prediction results (example: confusion matrix)
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
conf_mat = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
